### WIKIPEDIA RETRIVER

In [1]:
from langchain_community.retrievers import WikipediaRetriever

## Initialize the retriver (optional : set language and k)
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [6]:

# Define your query
query = "the man who died standing in football, Roberto Baggio"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [7]:
docs

[Document(metadata={'title': 'Roberto Baggio', 'summary': 'Roberto Baggio  (Italian pronunciation: [roˈbɛrto ˈbaddʒo]; born 18 February 1967) is an Italian former professional footballer who mainly played as a second striker, or as an attacking midfielder, although he was capable of playing in several offensive positions. He is the former president of the technical sector of the Italian Football Federation. A technically gifted creative playmaker and set piece specialist, renowned for his curling free-kicks, dribbling skills, and goalscoring, Baggio is widely regarded as one of the greatest players of all time.\nIn 1999, Baggio came fourth in the FIFA Player of the Century internet poll, and was chosen on the FIFA World Cup Dream Team in 2002. In 1993, he was named FIFA World Player of the Year and won the Ballon d\'Or. In 2004, he was named by Pelé in the FIFA 100, a list of the world\'s greatest living players.\nBaggio played for Italy in 56 matches and is the joint fourth-highest go

In [8]:
# Print retrieved content
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content}...")  # truncate for display


--- Result 1 ---
Content:
Roberto Baggio  (Italian pronunciation: [roˈbɛrto ˈbaddʒo]; born 18 February 1967) is an Italian former professional footballer who mainly played as a second striker, or as an attacking midfielder, although he was capable of playing in several offensive positions. He is the former president of the technical sector of the Italian Football Federation. A technically gifted creative playmaker and set piece specialist, renowned for his curling free-kicks, dribbling skills, and goalscoring, Baggio is widely regarded as one of the greatest players of all time.
In 1999, Baggio came fourth in the FIFA Player of the Century internet poll, and was chosen on the FIFA World Cup Dream Team in 2002. In 1993, he was named FIFA World Player of the Year and won the Ballon d'Or. In 2004, he was named by Pelé in the FIFA 100, a list of the world's greatest living players.
Baggio played for Italy in 56 matches and is the joint fourth-highest goalscorer for his national team. He s

### VectorStoreRetriver

In [9]:
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

In [10]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [11]:
# Step 2: Initialize embedding model
embedding_model = OllamaEmbeddings(model="gemma:2b")

# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [12]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [13]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [14]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


In [15]:
results = vectorstore.similarity_search(query, k=2)

In [16]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


### MMR

In [17]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [18]:
from langchain_community.vectorstores import FAISS

# Initialize OpenAI embeddings
# embedding_model = OpenAIEmbeddings()
embedding_model = OllamaEmbeddings(model="gemma:2b")

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [19]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance (0 - 1) 0 -> very diverse result and 1-> like other retrivers
)

In [20]:
query =  "what is langchain?"
results = retriever.invoke(query)

In [21]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
MMR helps you get diverse results when doing similarity search.

--- Result 3 ---
Embeddings are vector representations of text.


### Multiquery Retriver

In [47]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
print("MultiQueryRetriever imported successfully!")


MultiQueryRetriever imported successfully!


In [48]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [49]:

## embedding model
embedding_model = OllamaEmbeddings(model="gemma:2b")


# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [50]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [51]:
from langchain_groq import ChatGroq
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
# model = ChatGroq(model="openai/gpt-oss-120b")

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    # llm=ChatOpenAI(model="gpt-3.5-turbo")
    llm = ChatGroq(model="openai/gpt-oss-120b")
)

In [52]:
# Query
query = "How to improve energy levels and maintain balance?"

In [53]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [54]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 2 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 3 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 5 ---
The solar energy system in modern homes helps balance electricity demand.
******************************************************************************************************************************************************

--- Result 1 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 2 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 3 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and 

#### Contextual Compression Retrivers

In [56]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [57]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [58]:
# Create a FAISS vector store from the documents
# embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

In [ ]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) ### normal similarity search retriver

In [ ]:
# Set up the compressor using an LLM
# llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatGroq(model="openai/gpt-oss-120b")
compressor = LLMChainExtractor.from_llm(llm) ### creating a compressor

In [61]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [62]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [63]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis does not occur in animal cells.

--- Result 2 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.


**https://docs.langchain.com/oss/python/integrations/retrievers**